In [ ]:
! pip install datasets tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.3 MB/s 
     |████████████████████████████████| 115 kB 49.6 MB/s 
     |████████████████████████████████| 212 kB 47.0 MB/s 
     |████████████████████████████████| 182 kB 39.1 MB/s 
     |████████████████████████████████| 127 kB 41.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import collections
import random
import os
import time
import json
from PIL import Image
import io
import urllib
import uuid
from concurrent.futures import ThreadPoolExecutor
from functools import partial

import numpy as np
from tqdm import tqdm
from datasets import load_dataset
from datasets.utils.file_utils import get_datasets_user_agent
import matplotlib.pyplot as plt
import tensorflow as tf
from datasets import load_dataset

In [ ]:
! rm -rf images
! mkdir images

USER_AGENT = get_datasets_user_agent()

def fetch_single_image(image_url, timeout=None, retries=0):
    request = urllib.request.Request(
        image_url,
        data=None,
        headers={"user-agent": USER_AGENT},
    )
    with urllib.request.urlopen(request, timeout=timeout) as req:
        if 'png' in image_url:
          png = Image.open(io.BytesIO(req.read())).convert('RGBA')
          png.load() # required for png.split()
          background = Image.new("RGB", png.size, (255, 255, 255))
          background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
          background.resize((149,149))
          image_id = str(uuid.uuid4())
          image_path = "images/" + image_id + ".jpg"
          background.save(image_path, 'JPEG', quality=80)
        else:
          image = Image.open(io.BytesIO(req.read()))
          image.resize((149,149))
          image_id = str(uuid.uuid4())
          image_path = "images/" + image_id + ".jpg"
          image.save(image_path)
    return image_path

def fetch_images(batch, num_threads, timeout=None, retries=3):
    fetch_single_image_with_args = partial(fetch_single_image, timeout=timeout, retries=retries)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        batch["image_path"] = list(executor.map(fetch_single_image_with_args, batch["image_url"]))
    return batch



In [ ]:
df_test = load_dataset("csv", data_files='test.csv', download_mode='force_redownload')
df_test = df_test.filter(lambda f:f['ISO639-3'] == 'hau')

num_threads = 20
df_test = df_test.map(fetch_images, batched=True, batch_size=100, fn_kwargs={"num_threads": num_threads})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ae978c4a7c23889e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
df_test

DatasetDict({
    train: Dataset({
        features: ['Id', 'image_url', 'ISO639-3', 'image_path'],
        num_rows: 67
    })
})

In [ ]:
pip install azure-cognitiveservices-vision-computervision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.2 MB/s 
     |████████████████████████████████| 172 kB 31.8 MB/s 
     |████████████████████████████████| 41 kB 571 kB/s 


In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

region = 'eastus'
key = '2fa1e04ff2124cb99129d6a9638d6152'

credentials = CognitiveServicesCredentials(key)
client = ComputerVisionClient(
    endpoint="https://" + region + ".api.cognitive.microsoft.com/",
    credentials=credentials
)

In [ ]:
def cap_image(image):
  tags = client.describe_image_in_stream(image)
  return tags.captions[0].text

In [ ]:
prediction = {}
i=0
for image_path in tqdm(df_test['train']['image_path']):
    try:
      local_image = open(image_path, "rb")
      prediction[df_test['train']['Id'][i]] = (cap_image(local_image))
    except:
      print('\nwaiting\n')
      time.sleep(1)
    i = i+1


100%|██████████| 67/67 [01:04<00:00,  1.04it/s]


In [ ]:
import requests, uuid, json

# Add your key and endpoint
key = "7a93535c1e4548218adc1e77b43d994b"
endpoint = "https://api.cognitive.microsofttranslator.com"

# location, also known as region.
# required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
location = "eastus"

path = '/translate'
constructed_url = endpoint + path

headers = {
    'Ocp-Apim-Subscription-Key': key,
    # location required if you're using a multi-service or regional (not global) resource.
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

In [ ]:
params = {
    'api-version': '3.0',
    'from': ['en'],
    'to': ['ky']
}

body = []
for x in enumerate(prediction):
  body.append({'text': prediction[x]})

request = requests.post(constructed_url, params=params, headers=headers, json=body)
response = request.json()

In [12]:
! npm install --save google-translate-api

npm WARN deprecated uuid@2.0.3: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/math-random for details.
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ google-translate-api@2.3.0
added 32 packages from 14 contributors and audited 32 packages in 4.241s

3 packages are looking for funding
  run `npm fund` for details

found 7 vulnerabilities (2 moderate, 3 high, 2 critical)
  run `npm audit fix` to fix them, or `npm audit` for details


In [8]:
from googletrans import Translator
translator = Translator(service_urls='translate.google.com')

translations = translator.translate(['The quick brown fox', 'jumps over', 'the lazy dog'], dest='ko')
for translation in translations:
  print(translation.origin, ' -> ', translation.text)

ConnectError: ignored

In [10]:
from googletrans import Translator
translator = Translator()

translate_channel = translator.translate('Canal La Tele Perú', src='es', dest='en')
translate_channel2 = translator.translate('La defensa y las acciones ofensivas de Alex Dujshebaev dan a Españasu cuarto bronce en unos Juegos tras los de Atlanta 1996, Sydney 2000 y Pekín 2008.', src='es', dest='en')
print(translate_channel)
print(translate_channel2)

AttributeError: ignored

In [ ]:
captions_hau = []

for _ in tqdm(response):
  captions_hau.append(_['translations'][0]['text'])


100%|██████████| 67/67 [00:00<00:00, 247593.28it/s]


In [ ]:
import pandas as pd

submission = pd.DataFrame(list(zip(df_test['train']['Id'], captions_hau)), columns = ['Id', 'Predicted'])
submission.shape

submission.to_csv('hau_azure_translation.csv', index=0)